# Dynamically route logic based on input

This notebook covers how to do routing in the LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

There are two ways to perform routing:

    Using a RunnableBranch.
    Writing custom factory function that takes the input of a previous step and returns a runnable. Importantly, this should return a runnable and NOT actually execute.

We’ll illustrate both methods using a two step sequence where the first step classifies an input question as being about LangChain, Anthropic, or Other, then routes to a corresponding prompt chain.

# Using a RunnableBranch

A RunnableBranch is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it’s invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Here’s an example of what it looks like in action:

In [7]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms.minimax_llm import MiniMaxLLM

model = MiniMaxLLM()

In [8]:
# from langchain.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

First, let’s create a chain that will identify incoming questions as being about LangChain, Anthropic, or Other:

In [9]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

In [10]:
chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

Now, let’s create three sub chains:

In [11]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)

In [12]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [13]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [14]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, to use Anthropic, you first need to create an account on their website. Once you have an account, you can access their various tools and features. Anthropic's primary focus is on AI research, so if you're interested in AI, you can use their platform to learn more about the technology, participate in discussions, and contribute to the development of AI.")

In [15]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content="As Harrison Chase told me, to use LangChain, you first need to install the necessary libraries and dependencies, which can be done using pip or conda. Once installed, you can initialize a LangChain object and connect it to a search engine, such as OpenAI's API. You can then use the LangChain object to perform various tasks, such as question-answering, summarization, and text generation. It's important to note that LangChain is designed to work with OpenAI's API, so you will need an API key in order to use it.")

In [16]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='4')